
# Replacing Fundamental Modes.

As seen in the interactive notebook, as the thickness of the outer air layer of N2 increases, a lower mode comes and bumps out the fundamental.  In this notebook we visualize the modes as this occurs.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cmasher as cmr

from fiberamp.fiber.microstruct.bragg import BraggExact
from step_exact import plotlogf, plotlogf_real
from ngsolve.webgui import Draw
from ngsolve import CF
from scipy.optimize import newton
from ipywidgets import interactive, FloatSlider

importing NGSolve-6.2.2203-56-g47f483033


## Propagation Constant Replacement

Below we again give the interactive plot showing the replacement behavior.

In [2]:
d = 4.0775e-05  # thickness of innermost region (core radius)
ts = [d, 1e-5, d, 1.5e-5]

A = BraggExact(ts=ts, no_mesh=True)


In [3]:
nu = 1

def det_plot(T):
    d = 4.0775e-05  # thickness of innermost region (core radius)
    ts = [d, 1e-5, T * d, 1.5e-5]
    A.ts = ts
    cutoff = A.ks[0] * A.scale
    
    Xr = np.linspace(.99985* cutoff , .999874*cutoff , num=75)
    Xi = np.linspace( -.0002, .0001, num=75)
    xr, xi = np.meshgrid(Xr, Xi)
    zs = xr + 1j * xi
    
    fig, ax1 = plt.subplots(1, 1, figsize=(12,8))
    
    fs1 = A.determinant(zs, nu=nu, outer='h2')
    ax1.contour(xr, xi, np.log(np.abs(fs1)), levels=75)
    ax1.grid(True)
    ax1.set_facecolor('grey')
    ax1.set_title('Behavior near Fundamental Mode as outer thickness changes.')
#     ax1.set_xticklabels([])
#     ax1.set_yticklabels([])
    plt.show()
    
interactive_plot = interactive(det_plot, T=FloatSlider(min=1.26,max=1.376, step=.004,
                                                       value=1.376, readout_format='.3f'))
output = interactive_plot.children[-1]
output.layout.height = '10'
interactive_plot

interactive(children=(FloatSlider(value=1.376, description='T', max=1.376, min=1.26, readout_format='.3f', ste…

## Find the beta values for these two modes for the Ts in the slider range above.

In [4]:
Ts = np.arange(1.26, 1.376, .002)
Ts

array([1.26 , 1.262, 1.264, 1.266, 1.268, 1.27 , 1.272, 1.274, 1.276,
       1.278, 1.28 , 1.282, 1.284, 1.286, 1.288, 1.29 , 1.292, 1.294,
       1.296, 1.298, 1.3  , 1.302, 1.304, 1.306, 1.308, 1.31 , 1.312,
       1.314, 1.316, 1.318, 1.32 , 1.322, 1.324, 1.326, 1.328, 1.33 ,
       1.332, 1.334, 1.336, 1.338, 1.34 , 1.342, 1.344, 1.346, 1.348,
       1.35 , 1.352, 1.354, 1.356, 1.358, 1.36 , 1.362, 1.364, 1.366,
       1.368, 1.37 , 1.372, 1.374])

We can graph the real part and look for places it dips to get guesses for the roots

In [5]:
nu = 1

def det_plot_real(T):
    d = 4.0775e-05                 # thickness of innermost region (core radius)
    ts = [d, 1e-5, T * d, 1.5e-5]  # d is base thickness, scaled by input T
    A.ts = ts                      # set ts in BraggExact class (it updates necessary derived values)
    cutoff = A.ks[0] * A.scale     # mode cutoff, used for limits
    
    xr = np.linspace(.999847* cutoff , .999874*cutoff , num=1000)
    h = xr[1] - xr[0]                         # h for estimating derivatives
    fs = A.determinant(xr, nu=nu, outer='h2') # function values
    fsa = np.abs(fs)                          # abs of values
    fsp = (fsa[2:] - fsa[:-2])/(2*h)          # center difference estimate for derivative of abs(fs)
    
    fspp = (fsa[2:] - 2 * fsa[1:-1] + fsa[:-2])/(h**2)  # fourth order accurate second derivative difference estimate

    dp = fsp[1:] * fsp[:-1]         # find where sign of derivative changes
    indices = np.where(dp <= 0)[0]  # great idea from stack overflow: https://stackoverflow.com/questions/45174182/finding-all-roots-in-a-given-interval-in-1d


    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12,12))
    ax1.plot(xr, np.log(fsa))
    ax2.plot(xr[1:-1],fsp, [xr[1], xr[-2]], [0,0])   # derivatives exist at interior points xr[1:-1]
#     ax3.plot(xr[1:-1],fspp, [xr[1], xr[-2]], [0,0])
    
    for idx in indices:  # graph vertical lines at zeros of derivative
        if fspp[idx] < 0:  # filter out local maxes using 2nd derivative
            pass
        else:
            m, M = ax1.get_ylim()
            ax1.plot([xr[idx], xr[idx]], [m,M])
            m, M = ax2.get_ylim()
            ax2.plot([xr[idx], xr[idx]], [m,M])

    ax1.grid('both', axis='both')
    ax2.grid('both', axis='both')
#     ax3.grid('both', axis='both')
    ax1.set_title('Finding propagation constants as replacement occurs.\n')
    plt.show()
    
interactive_plot = interactive(det_plot_real, T=FloatSlider(min=1.26,max=1.376, step=.002,
                                                       value=1.376, readout_format='.3f'))
output = interactive_plot.children[-1]
output.layout.height = '10'
interactive_plot

interactive(children=(FloatSlider(value=1.376, description='T', max=1.376, min=1.26, readout_format='.3f', ste…

## Run the cell below to get the beta values

In [ ]:
nu = 1
outer = 'h2'
betas = np.zeros((len(Ts), 2), dtype=complex)

for i,T in enumerate(Ts):
    d = 4.0775e-05                 # thickness of innermost region (core radius)
    ts = [d, 1e-5, T * d, 1.5e-5]  # d is base thickness, scaled by input T
    A.ts = ts                      # set ts in BraggExact class (it updates necessary derived values)
    cutoff = A.ks[0] * A.scale     # mode cutoff, used for limits

    xr = np.linspace(.999847* cutoff , .999874*cutoff , num=1000)
    h = xr[1] - xr[0]                         # h for estimating derivatives
    fs = A.determinant(xr, nu=nu, outer='h2') # function values
    fsa = np.abs(fs)                          # abs of values
    fsp = (fsa[2:] - fsa[:-2])/(2*h)          # center difference estimate for derivative of abs(fs)

    fspp = (fsa[2:] - 2 * fsa[1:-1] + fsa[:-2])/(h**2)  # fourth order accurate second derivative difference estimate

    dp = fsp[1:] * fsp[:-1]         # find where sign of derivative changes
    indices = np.where(dp <= 0)[0]  # great idea from stack overflow: https://stackoverflow.com/questions/45174182/finding-all-roots-in-a-given-interval-in-1d
    j=0
    print(i)
    for idx in indices:  # graph vertical lines at zeros of derivative
        if fspp[idx] < 0:  # filter out local maxes using 2nd derivative
            pass
        else:
            guess = np.array(xr[idx])
            betas[i,j] = newton(A.determinant, guess, args=(nu, outer), tol = 1e-15)
            j+=1
    

## Visualize the modes

We need this to be quicker than the typical ngsolve graphing.  I did implement a matplot version I'll now bring back.

In [8]:
N_T, N, M = len(Ts), 100, 100
thetas = np.linspace(0, 2*np.pi, M)
cmap = cmr.get_sub_cmap('jet', 0.2, 0.89)

Z = np.zeros((N_T, 2, N, M), dtype=complex)
A = BraggExact(ts=ts)
Ts = np.arange(1.26, 1.376, .002)


In [ ]:

for i, T in enumerate(Ts):
    d = 4.0775e-05                 # thickness of innermost region (core radius)
    ts = [d, 1e-5, T * d, 1.5e-5]  # d is base thickness, scaled by input T
    A.ts = ts
    
    R = A.rhos[-1] / A.scale
    rs = np.linspace(0, R, N)
    Rs, Thetas = np.meshgrid(rs, thetas)

    X, Y = Rs * np.cos(Thetas), Rs * np.sin(Thetas)
    for j in [0,1]:
        Etv = (A.all_fields(betas[i,j], nu=nu, outer=outer)['Etv']).Norm()
        print(i,j)
        for k in range(N):
            for s in range(M):  
                pt = A.mesh(X[k,s],Y[k,s])
                Z[i, j, k, s] = Etv(pt)



## Almost ready

In [ ]:
# from IPython.display import display, HTML
# display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
# np.save('Zs', Z)

In [10]:
d = 4.0775e-05  # thickness of innermost region (core radius)
ts = [d, 1e-5, d, 1.5e-5]

A = BraggExact(ts=ts, no_mesh=True)
Z = np.load('data/Zs.npy')

In [11]:
nu = 1
Ts = np.arange(1.26, 1.376, .002)
d = 4.0775e-05  # thickness of innermost region (core radius)
ts = [d, 1e-5, Ts[-1] * d, 1.5e-5]
A.ts = ts

bigR = A.rhos[-1]

def det_plot(T):
    idx = np.where(Ts==T)[0][0]
    d = 4.0775e-05  # thickness of innermost region (core radius)
    ts = [d, 1e-5, T * d, 1.5e-5]
    A.ts = ts
    
    R = A.rhos[-1]

    rs = np.linspace(0, R, N)
    Rs, Thetas = np.meshgrid(rs, thetas)

    X, Y = Rs * np.cos(Thetas), Rs * np.sin(Thetas)
    
    cutoff = A.ks[0] * A.scale
    
    Xr = np.linspace(.999847* cutoff , .999874*cutoff , num=75)
    Xi = np.linspace( -.00015, .000075, num=75)
    xr, xi = np.meshgrid(Xr, Xi)
    zs = xr + 1j * xi
    
    fig = plt.figure(figsize=(17,14))
    gs = fig.add_gridspec(2, 2, wspace=0.1, width_ratios=[1,1],
                         height_ratios=[1.1,1.85])
    
    ax1 = fig.add_subplot(gs[0, :])
    ax2 = fig.add_subplot(gs[1, 0])
    ax3 = fig.add_subplot(gs[1, 1])
    
    fs1 = A.determinant(zs, nu=nu, outer='h2')
    ax1.contour(xr, xi, np.log(np.abs(fs1)), levels=75)
    ax1.grid(True)
    ax1.set_facecolor('grey')
    ax1.set_title('Beta Behavior near Fundamental Mode as outer thickness changes.\n', fontsize=20)
#     ax1.set_yticklabels([])
#     ax1.set_xticklabels([])
    ax1.set_yticks([0])
    ax1.set_xticks([])

    
    ax2.contourf(X, Y, Z[idx, 0,:,:], levels=30, cmap=cmap)
    ax2.set_xlim(-bigR, bigR)
    ax2.set_ylim(-bigR, bigR)
#     ax2.axis('square')
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax2.set_frame_on(False)
    ax2.set_title('Left Mode', fontsize=16)

    ax3.contourf(X, Y, Z[idx, 1,:,:], levels=30, cmap=cmap)
    ax3.set_xlim(-bigR, bigR)
    ax3.set_ylim(-bigR, bigR)
#     ax3.axis('equal')
    ax3.set_frame_on(False)
    ax3.set_title('Right Mode', fontsize=16)
    ax3.set_xticks([])
    ax3.set_yticks([])
    plt.show()
    
interactive_plot = interactive(det_plot, T=FloatSlider(min=1.26,max=1.356, step=.002,
                                                       value=1.26, readout_format='.3f'))
output = interactive_plot.children[-1]
output.layout.height = '10'
interactive_plot

interactive(children=(FloatSlider(value=1.26, description='T', max=1.356, min=1.26, readout_format='.3f', step…